In [367]:
import pandas as pd
import numpy as np
import math
import re
import seaborn as sns
import matplotlib as plt
from matplotlib import pyplot as plt

In [368]:
from sklearn.neighbors import KNeighborsRegressor

In [369]:
test =pd.read_csv('data/test.csv', index_col=0)
test_forest = pd.read_csv('data/test.csv', index_col=0)
train =pd.read_csv('data/train.csv', index_col=0)

In [370]:
#check for missingness
test[test.isna().any(axis=1)]

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
Id,,,,,,,,,,,,,,,,,,,,,
1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,Inside,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,Inside,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal
1466,60,RL,75.0,10000,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,0,NaN,NaN,NaN,0,4,2010,WD,Normal
1467,20,RL,NaN,7980,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,GdPrv,Shed,500,3,2010,WD,Normal
1468,60,RL,63.0,8402,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,5,2010,WD,Normal
1469,20,RL,85.0,10176,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,2,2010,WD,Normal


In [371]:
test.drop(list(test[test['MasVnrArea'].isnull()].index), inplace=True)
test_forest.drop(list(test_forest[test_forest['MasVnrArea'].isnull()].index), inplace=True)

In [372]:
test[test['LotConfig'].isnull()]

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
Id,,,,,,,,,,,,,,,,,,,,,


In [373]:
test['LotConfig'].unique()

array(['Inside', 'Corner', 'FR2', 'CulDSac', 'FR3'], dtype=object)

In [375]:
train_frontage = train[~train['LotFrontage'].isnull()][['LotFrontage', 'LotArea', 'LotConfig']]
train_missing = train[train['LotFrontage'].isnull()][['LotFrontage', 'LotArea', 'LotConfig']]
imputed_df = pd.DataFrame(columns=['LotFrontage', 'LotArea', 'LotConfig'])

for config in train['LotConfig'].unique():
    complete_subset = train_frontage[train_frontage['LotConfig'] == config]
    missing_subset = train_missing[train_missing['LotConfig'] == config]
    k = int(np.sqrt(complete_subset.shape[0] + missing_subset.shape[0]))
    clf = KNeighborsRegressor(k, weights='distance')
    trained_model = clf.fit(complete_subset['LotArea'].values.reshape(-1, 1), 
                            complete_subset['LotFrontage'])
    if missing_subset['LotArea'].values.reshape(-1, 1).shape[0] > 0:
        imputed_values = trained_model.predict(missing_subset['LotArea'].values.reshape(-1, 1))
        missing_subset['LotFrontage'] = imputed_values.reshape(-1)
    imputed_df = pd.concat([imputed_df,complete_subset,missing_subset])
imputed_df = imputed_df.sort_index()
test['LotFrontage'] = imputed_df['LotFrontage']
test['LotFrontage']

/Users/zhuwang/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/zhuwang/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/zhuwang/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pan

Id
1461   NaN
1462   NaN
1463   NaN
1464   NaN
1465   NaN
1466   NaN
1467   NaN
1468   NaN
1469   NaN
1470   NaN
1471   NaN
1472   NaN
1473   NaN
1474   NaN
1475   NaN
1476   NaN
1477   NaN
1478   NaN
1479   NaN
1480   NaN
1481   NaN
1482   NaN
1483   NaN
1484   NaN
1485   NaN
1486   NaN
1487   NaN
1488   NaN
1489   NaN
1490   NaN
        ..
2890   NaN
2891   NaN
2892   NaN
2893   NaN
2894   NaN
2895   NaN
2896   NaN
2897   NaN
2898   NaN
2899   NaN
2900   NaN
2901   NaN
2902   NaN
2903   NaN
2904   NaN
2905   NaN
2906   NaN
2907   NaN
2908   NaN
2909   NaN
2910   NaN
2911   NaN
2912   NaN
2913   NaN
2914   NaN
2915   NaN
2916   NaN
2917   NaN
2918   NaN
2919   NaN
Name: LotFrontage, Length: 1444, dtype: float64

In [323]:
temp=list(test[test['BsmtExposure'].isnull() & test['BsmtQual'].notnull()].index)
#1488
#2349

In [324]:
import random
random.seed(0)
#obs1488 and 2349 is randomly assigned a a non-"No" categery for BsmtExposure
test['BsmtExposure'][1488]=test[test['BsmtExposure']!='No'].sample()['BsmtExposure']
test['BsmtExposure'][2349]=test['BsmtExposure'][1488]

/Users/zhuwang/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Users/zhuwang/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [325]:
#ob with a BsmtFinSF2 but shows "no basement" in the BsmtFinType2 col NEED TO IMPUTE  - random
test[test['BsmtFinType2'].isnull() & test['BsmtFinSF2']!=0]

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
Id,,,,,,,,,,,,,,,,,,,,,


In [326]:
# no need to impute BsmtFinType2, no missing obs in test set 
# random.seed(0)
# #obs333 is randomly assigned a non-"No" categery for BsmtFinType2
# train['BsmtFinType2'][333]=train[train['BsmtFinType2']!='nan'].sample()['BsmtFinType2'].values[0]

In [327]:
#ob with Na in Electrical 
test[test['Electrical'].isnull()]

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
Id,,,,,,,,,,,,,,,,,,,,,


In [328]:
# no need to impute Electrical, no missing obs in test set 
# random.seed(0)
# #obs1380 is randomly assigned a random
# train['Electrical'][1380] = train['Electrical'].sample().values[0]

In [329]:
#for BsmtExposure - replace No with NoExposure; replace NaN with NoBsmt
test['BsmtExposure']=test['BsmtExposure'].str.replace('No','NoExposure')

In [330]:
test['BsmtExposure'].replace(np.NaN,'NoBsmt',inplace=True)

In [331]:
#GarageYrBlt - change No to numerical zero 
test['GarageYrBlt'].replace(np.NaN,0,inplace=True)

In [332]:
#cols for which NA will be replaced with "No" because NA is a true category thus need to change 
#to "No" so Pandas does not pick it up as np.NaN
#replacing fixing NA from only categorical Nominal
FixNA=['Alley', 'GarageType', 'MiscFeature']

In [333]:
#change np.NaN to "No"
for v in FixNA:
    test[v].replace(np.NaN,'No',inplace=True)

In [334]:
misc=test[['MiscFeature','MiscVal']]
misc=pd.pivot_table(misc,index='Id',columns='MiscFeature',values='MiscVal')
misc.replace(np.NaN,0,inplace=True)
misc=misc.drop(['No','Othr'],axis=1).apply(pd.to_numeric, errors='ignore', downcast='integer')

In [335]:
#combined misc with original DF and drop MiscFeature, MiscVal
test = pd.concat([test.drop(['MiscFeature','MiscVal'],axis=1),misc],axis=1)

In [336]:
#list of categorical variables 
cat_variables=['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour','Utilities',
               'LotConfig', 'LandSlope','Neighborhood', 'Condition1', 'Condition2',
               'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl','Exterior1st', 
               'Exterior2nd', 'MasVnrType', 'Foundation','BsmtExposure','BsmtFinType2',
               'Heating','CentralAir','Electrical', 'GarageType','SaleType', 'SaleCondition']


In [337]:
for v in cat_variables:
    print(test[v].value_counts())

RL         1103
RM          242
FV           70
C (all)      15
RH           10
Name: MSZoning, dtype: int64
Pave    1438
Grvl       6
Name: Street, dtype: int64
No      1338
Grvl      70
Pave      36
Name: Alley, dtype: int64
Reg    928
IR1    476
IR2     34
IR3      6
Name: LotShape, dtype: int64
Lvl    1297
HLS      69
Bnk      54
Low      24
Name: LandContour, dtype: int64
AllPub    1442
Name: Utilities, dtype: int64
Inside     1072
Corner      244
CulDSac      81
FR2          37
FR3          10
Name: LotConfig, dtype: int64
Gtl    1381
Mod      60
Sev       3
Name: LandSlope, dtype: int64
NAmes      218
OldTown    126
CollgCr    116
Edwards     93
Somerst     89
NridgHt     88
Gilbert     82
Sawyer      77
SawyerW     66
Mitchel     65
NWAmes      58
IDOTRR      56
Crawfor     52
BrkSide     50
Timber      33
NoRidge     30
StoneBr     26
SWISU       23
MeadowV     20
ClearCr     16
BrDale      14
NPkVill     14
Veenker     13
Blmngtn     11
Blueste      8
Name: Neighborhood, dtyp

In [338]:
feature_drop=['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour','Utilities',
               'LandSlope','BldgType', 'HouseStyle','RoofStyle', 'RoofMatl',
              'BsmtFinType2','Heating','Electrical','Condition1', 'Condition2',
               'CentralAir','GarageType','SaleType','BsmtFinSF1','BsmtFinSF2','BsmtUnfSF',
             '1stFlrSF','2ndFlrSF','LowQualFinSF']

#drop categorical features that present sparsity 
#MSZoning, Street, Alley, LotShape, LandCounter, Utilities, LandSlope, Condition 1,
#Condition 2, Roofstyle, RoofMatl, HouseStyle, #Heating #electrocial, SaleType
#centralair

#drop continuous variables that are redundant with total sqrt footage 
#BsmtFinSF2
#BsmtUnSF
#1stFlrSF
#2ndFlrSF
#LowQualFinSF
test=test.drop(feature_drop,axis=1)

In [339]:
train['PriceSqFt'] = train['SalePrice'] / train['GrLivArea']
hoodscore = train.groupby('Neighborhood').agg({'PriceSqFt': 'mean'}).sort_values(by='PriceSqFt', ascending=True).rank()
hoodscore['PriceSqFt']=hoodscore['PriceSqFt'].astype('int64')
#neighborhoods = avg_price_neighborhood.index.to_numpy()
#neighbscore=pd.DataFrame()
#train['Neighborhood'] = train['Neighborhood'].apply(lambda x: np.where(neighborhoods == x)[0][0] + 1)
#train = train.drop('PriceSqFt', axis=1)
#train['Neighborhood']
type(hoodscore)


pandas.core.frame.DataFrame

In [340]:
test=test.merge(hoodscore, left_on='Neighborhood', right_index=True)
test.columns

Index(['MSSubClass', 'LotFrontage', 'LotArea', 'LotConfig', 'Neighborhood',
       'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'Exterior1st', 'Exterior2nd', 'MasVnrType', 'MasVnrArea', 'ExterQual',
       'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure',
       'BsmtFinType1', 'TotalBsmtSF', 'HeatingQC', 'GrLivArea', 'BsmtFullBath',
       'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr',
       'KitchenQual', 'TotRmsAbvGrd', 'Functional', 'Fireplaces',
       'FireplaceQu', 'GarageYrBlt', 'GarageFinish', 'GarageCars',
       'GarageArea', 'GarageQual', 'GarageCond', 'PavedDrive', 'WoodDeckSF',
       'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea',
       'PoolQC', 'Fence', 'MoSold', 'YrSold', 'SaleCondition', 'Gar2', 'Shed',
       'PriceSqFt'],
      dtype='object')

In [341]:
test.drop('Neighborhood',axis=1,inplace=True)
test.columns

Index(['MSSubClass', 'LotFrontage', 'LotArea', 'LotConfig', 'OverallQual',
       'OverallCond', 'YearBuilt', 'YearRemodAdd', 'Exterior1st',
       'Exterior2nd', 'MasVnrType', 'MasVnrArea', 'ExterQual', 'ExterCond',
       'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1',
       'TotalBsmtSF', 'HeatingQC', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath',
       'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond', 'PavedDrive', 'WoodDeckSF', 'OpenPorchSF',
       'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'PoolQC',
       'Fence', 'MoSold', 'YrSold', 'SaleCondition', 'Gar2', 'Shed',
       'PriceSqFt'],
      dtype='object')

In [342]:
test.rename(columns={'PriceSqFt':'Neighborhood'}, inplace=True)
test.columns

Index(['MSSubClass', 'LotFrontage', 'LotArea', 'LotConfig', 'OverallQual',
       'OverallCond', 'YearBuilt', 'YearRemodAdd', 'Exterior1st',
       'Exterior2nd', 'MasVnrType', 'MasVnrArea', 'ExterQual', 'ExterCond',
       'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1',
       'TotalBsmtSF', 'HeatingQC', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath',
       'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond', 'PavedDrive', 'WoodDeckSF', 'OpenPorchSF',
       'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'PoolQC',
       'Fence', 'MoSold', 'YrSold', 'SaleCondition', 'Gar2', 'Shed',
       'Neighborhood'],
      dtype='object')

In [343]:
#Exterior1st - Assign other to categories with count under 100 
test['Exterior1st'].replace(['CemntBd','BrkFace','WdShing','Stucco','AsbShng','Stone',
                                     'BrkComm','AsphShn','CBlock','ImStucc'],'Other',inplace=True)


In [344]:
#Exterior2nd - Assign other to categories with count under 100 train['Exterior2nd'].replace(['CemntBd','BrkFace','WdShing','Stucco','AsbShng','Stone',

test['Exterior2nd'].replace(['CemntBd','Wd Shng','Stucco','BrkFace','AsbShng','Stone',
                                     'Brk Cmn','AsphShn','CBlock','ImStucc'],'Other',inplace=True)

In [345]:
test['Foundation'].replace(['Slab','Stone','Wood'],'Other',inplace=True)

In [346]:
LST = ('ExterQual','ExterCond','HeatingQC','KitchenQual')
LST1 = ('BsmtQual','BsmtCond','FireplaceQu','GarageQual','GarageCond')
LST2 = ('BsmtFinType1')
LST3 = ('Functional')
LST4 = ('GarageFinish')
LST5 = ('PavedDrive')
LST6 = ('PoolQC')
LST7 = ('Fence')


In [347]:
test['TotalBath']=test['BsmtFullBath']+test['BsmtHalfBath']+test['FullBath']+test['HalfBath']
test['Bedroom/Bathroom']=test['BedroomAbvGr']/test['TotalBath']
test['Bedroom/Bathroom'].unique()


array([2.        , 1.5       , 1.        , 0.66666667, 3.        ,
       1.33333333, 2.5       , 4.        , 1.66666667, 0.33333333,
       0.75      , 1.25      , 0.5       , 0.8       , 0.6       ,
       0.4       , 0.25      ,        nan, 0.        ])

In [348]:
for x in LST:
    test.loc[test[x] == 'Po',x] = 1
    test.loc[test[x] == 'Fa',x] = 2
    test.loc[test[x] == 'TA',x] = 3
    test.loc[test[x] == 'Gd',x] = 4
    test.loc[test[x] == 'Ex',x] = 5

In [349]:
for x in LST1:
    test[x].fillna(0, inplace=True)    
    test.loc[test[x] == 'Po',x] = 1
    test.loc[test[x] == 'Fa',x] = 2
    test.loc[test[x] == 'TA',x] = 3
    test.loc[test[x] == 'Gd',x] = 4
    test.loc[test[x] == 'Ex',x] = 5
    

/Users/zhuwang/anaconda3/lib/python3.7/site-packages/pandas/core/ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


In [350]:
# Nan values mean there are none for this there is no basement therefore value is 0
# all columns for ordinal has no basement or features if filled with 0
x = LST2
test[x].fillna(0, inplace=True)
test.loc[test[x] == 'Unf',x] = 1
test.loc[test[x] == 'LwQ',x] = 2
test.loc[test[x] == 'Rec',x] = 3
test.loc[test[x] == 'BLQ',x] = 4
test.loc[test[x] == 'ALQ',x] = 5
test.loc[test[x] == 'GLQ',x] = 6

In [351]:
x = LST3
test.loc[test[x] == 'Sal',x] = 1
test.loc[test[x] == 'Sev',x] = 2
test.loc[test[x] == 'Maj2',x] = 3
test.loc[test[x] == 'Maj1',x] = 4
test.loc[test[x] == 'Mod',x] = 5
test.loc[test[x] == 'Min2',x] = 6
test.loc[test[x] == 'Min1',x] = 6
test.loc[test[x] == 'Typ',x] = 6

In [352]:
x = LST4
test[x].fillna(0, inplace=True)
test.loc[test[x] == 'Unf',x] = 1
test.loc[test[x] == 'RFn',x] = 2
test.loc[test[x] == 'Fin',x] = 3

    

In [353]:
x = LST5
test.loc[test[x] == 'N',x] = 1
test.loc[test[x] == 'P',x] = 2
test.loc[test[x] == 'Y',x] = 3


In [354]:
x = LST6
test[x].fillna(0, inplace=True)
test.loc[test[x] == 'Fa',x] = 1
test.loc[test[x] == 'TA',x] = 2
test.loc[test[x] == 'Gd',x] = 3
test.loc[test[x] == 'Ex',x] = 4
    

In [355]:
x = LST7
test[x].fillna(0, inplace=True)
test.loc[test[x] == 'MnWw',x] = 1
test.loc[test[x] == 'GdWo',x] = 2
test.loc[test[x] == 'MnPrv',x] = 3
test.loc[test[x] == 'GdPrv',x] = 4    

In [356]:
# These are the ordinal columns of the training data set
# train[['OverallQual','OverallCond','ExterQual','ExterCond','BsmtQual','BsmtCond','BsmtFinType1','HeatingQC','BsmtFullBath','KitchenQual',
#       'Functional',
# 'FireplaceQu','GarageFinish','GarageQual','GarageCond','PavedDrive','PoolQC','Fence','SalePrice']]
#dropping bathroom columns since we made a new column Total Bathroom
test = test.drop('BsmtFullBath',1)
test = test.drop('BsmtHalfBath',1)
test = test.drop('FullBath',1)
test = test.drop('HalfBath',1)

In [357]:
test[test.isna().any(axis=1)]

,MSSubClass,LotFrontage,LotArea,LotConfig,OverallQual,OverallCond,YearBuilt,YearRemodAdd,Exterior1st,Exterior2nd,...,PoolQC,Fence,MoSold,YrSold,SaleCondition,Gar2,Shed,Neighborhood,TotalBath,Bedroom/Bathroom
Id,,,,,,,,,,,,,,,,,,,,,
1461,20,NaN,11622,Inside,5,6,1961,1961,VinylSd,VinylSd,...,0,3,6,2010,Normal,0,0,10,1.0,2.000000
1462,20,NaN,14267,Corner,6,6,1958,1958,Wd Sdng,Wd Sdng,...,0,0,6,2010,Normal,12500,0,10,2.0,1.500000
1470,20,NaN,8400,Corner,4,5,1970,1970,Plywood,Plywood,...,0,3,4,2010,Normal,0,0,10,2.0,1.000000
1471,120,NaN,5858,FR2,7,5,1999,1999,MetalSd,MetalSd,...,0,0,6,2010,Normal,0,0,10,3.0,0.666667
1510,20,NaN,9600,Inside,5,5,1966,1966,VinylSd,VinylSd,...,0,0,4,2010,Normal,0,0,10,3.0,1.000000
1511,20,NaN,9900,Corner,5,5,1966,1966,HdBoard,HdBoard,...,0,0,4,2010,Normal,0,0,10,2.0,1.500000
1512,20,NaN,9680,Inside,5,6,1967,1967,Wd Sdng,Plywood,...,0,4,4,2010,Normal,0,0,10,3.0,1.000000
1513,80,NaN,10600,Inside,6,5,1964,1964,HdBoard,HdBoard,...,0,0,5,2010,Family,0,0,10,2.0,1.500000
1514,90,NaN,13260,Inside,5,6,1962,2001,HdBoard,HdBoard,...,0,0,1,2010,Abnorml,0,0,10,4.0,1.500000


In [200]:
test= test.drop('MSSubClass', 1)
test

,LotFrontage,LotArea,LotConfig,Neighborhood,OverallQual,OverallCond,YearBuilt,YearRemodAdd,Exterior1st,Exterior2nd,...,Fence,MoSold,YrSold,SaleCondition,SalePrice,Gar2,Shed,TenC,TotalBath,Bedroom/Bathroom
Id,,,,,,,,,,,,,,,,,,,,,
1,65.0,8450,Inside,20,7,5,2003,2003,VinylSd,VinylSd,...,0,2,2008,Normal,208500,0,0,0,4,0.75
2,80.0,9600,FR2,23,6,8,1976,1976,MetalSd,MetalSd,...,0,5,2007,Normal,181500,0,0,0,3,1.00
3,68.0,11250,Inside,20,7,5,2001,2002,VinylSd,VinylSd,...,0,9,2008,Normal,223500,0,0,0,4,0.75
4,60.0,9550,Corner,15,7,5,1915,1970,Wd Sdng,Other,...,0,2,2006,Abnorml,140000,0,0,0,2,1.50
5,84.0,14260,FR2,18,8,5,2000,2000,VinylSd,VinylSd,...,0,12,2008,Normal,250000,0,0,0,4,1.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1456,62.0,7917,Inside,14,6,5,1999,2000,VinylSd,VinylSd,...,0,8,2007,Normal,175000,0,0,0,3,1.00
1457,85.0,13175,Inside,9,6,6,1978,1988,Plywood,Plywood,...,3,2,2010,Normal,210000,0,0,0,3,1.00
1458,66.0,9042,Inside,15,7,9,1941,2006,Other,CmentBd,...,4,5,2010,Normal,266500,0,2500,0,2,2.00


In [201]:
test.to_csv('data/test_clean_forest.csv')

In [202]:
cat_variables2=['LotConfig','Exterior1st', 'Exterior2nd', 'Foundation','MasVnrType',
               'SaleCondition','BsmtExposure']

In [203]:
#dummify categorical nominal and binary variables
for v in cat_variables2:
    temp = pd.get_dummies(test[v],prefix=v, prefix_sep='_')
    temp = temp.drop(temp.iloc[:,0:1],axis=1)
    test = pd.concat([train.drop(v,axis=1),temp],axis=1)

In [205]:
test.to_csv('data/test_clean.csv')